1. 单线图、多线图
2. 面积图、堆叠面积图

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [19]:
# 1、折线图 - 单线图

# 导入ColumnDataSource模块
# 将数据存储为ColumnDataSource对象
# 参考文档：http://bokeh.pydata.org/en/latest/docs/user_guide/data.html
# 可以将dict、Dataframe、group对象转化为ColumnDataSource对象

from bokeh.models import ColumnDataSource

# 转化为ColumnDataSource对象
# 这里注意了，index和columns都必须有名称字段
df = pd.DataFrame({'value': np.random.randn(100).cumsum()})
df.index.name = 'index'
source = ColumnDataSource(data=df)

In [20]:
p = figure(plot_width=800, plot_height=600)

p.line(x='index',
       y='value',
       source=source,
       line_width=1,
       line_alpha=0.8,
       line_color='black',
       line_dash=[10, 6])
p.circle(x='index', y='value', source=source, size=2, color='red', alpha=0.8)

show(p)

In [21]:
# 1、折线图 - 多线图
# ① multi_line

# 创建数据
df = pd.DataFrame({
    'A': np.random.randn(100).cumsum(),
    "B": np.random.randn(100).cumsum()
})

In [22]:
p = figure(plot_width=800, plot_height=600)

p.multi_line([df.index, df.index], [df['A'], df['B']],
             color=['firebrick', 'navy'],
             alpha=[0.8, 0.6],
             line_width=[2, 1])

show(p)

In [23]:
# 1、折线图 - 多线图
# ② 多个line

x = np.linspace(0.1, 5, 100)

In [24]:
# 这里设置对数坐标轴
p = figure(title='log axis example', y_axis_type='log', y_range=(0.001, 10**22))

p.line(x,
       np.sqrt(x),
       legend='y=sqrt(x)',
       line_color='tomato',
       line_dash='dotdash')

p.line(x, x, legend='y=x')
p.circle(x, x, legend='y=x')

p.line(x, x**2, legend='y=x^2')
p.circle(x, x**2, legend='y=x^2', fill_color=None, line_color='olivedrab')

p.line(x, 10**2, legend='y=10^x', line_color='gold', line_width=2)

p.line(x,
       x**x,
       legend='y=x^x',
       line_dash='dotted',
       line_color='indigo',
       line_width=2)

p.line(x,
       10**(x**2),
       legend='y=10^(x^2)',
       line_color='coral',
       line_dash='dashed',
       line_width=2)

p.legend.location = 'top_left'
p.xaxis.axis_label = 'Domain'
p.yaxis.axis_label = 'Values (log scale)'

show(p)

In [25]:
# 2、面积图 - 单维度面积图

# 创建数据
# 注意设定起始值和终点值为最低点
s = pd.Series(np.random.randn(100).cumsum())
s.iloc[0] = 0
s.iloc[-1] = 0

In [26]:
p = figure(plot_width=600, plot_height=400)
# 绘制面积图
# .patch将会把所有点连接成一个闭合面
p.patch(s.index,
        s.values,
        line_width=1,
        line_alpha=0.8,
        line_color='black',
        line_dash=[10, 6],
        fill_color='black',
        fill_alpha=0.15)
p.circle(s.index, s.values, size=5, color='red', alpha=0.8)

show(p)

In [27]:
# 2、面积图 - 面积堆叠图

# 导入brewer模块
from bokeh.palettes import brewer

N = 20
cats = 10
rng = np.random.RandomState(1)
df = pd.DataFrame(rng.randint(10, 100, size=(N, cats))).add_prefix('y')

In [28]:
df.shape

(20, 10)

In [29]:
# 每一个堆叠面积图的最高点
df_top = df.cumsum(axis=1)
# 每一个堆叠面积图的最低点，并上下反向
df_bottom = df_top.shift(axis=1).fillna({'y0': 0})[::-1]  # 数据方向→ ↕
# 数据合并，每一组数据都是一个可以围合成一个面的散点集合
df_stack = pd.concat([df_top, df_bottom], ignore_index=True)

In [30]:
df_stack.shape

(40, 10)

In [31]:
# 根据变量数拆分颜色
colors = brewer['Spectral'][df_stack.shape[1]]
# 得到围合顺序的index，这里由于一列是20个元素，所以连接成面需要40个点
x = np.hstack((df.index[::-1], df.index))

In [32]:
p = figure(x_range=(0, N - 1), y_range=(0, 700))

# 一层两线地绘制
p.patches(
    [x] * df_stack.shape[1],  # 得到10组index
    [df_stack[c].values for c in df_stack],  # c为df_stack的列名，这里得到10组对应的valyes
    color=colors,  # 设置其他参数
    alpha=0.8,
    line_color=None)

show(p)

In [33]:
[x] * df_stack.shape[1]

[array([19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
         2,  1,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
        14, 15, 16, 17, 18, 19], dtype=int64),
 array([19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
         2,  1,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
        14, 15, 16, 17, 18, 19], dtype=int64),
 array([19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
         2,  1,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
        14, 15, 16, 17, 18, 19], dtype=int64),
 array([19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
         2,  1,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
        14, 15, 16, 17, 18, 19], dtype=int64),
 array([19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
         2,  1,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
        14, 15, 16, 17, 18, 19], dtype=int64),
 arra

In [34]:
[df_stack[c].values for c in df_stack]

[array([47., 86., 39., 73., 23., 13., 25., 77., 20., 98., 95., 86., 23.,
        75., 76., 32., 93., 25., 87., 55.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.]),
 array([ 69., 167.,  63., 144.,  80.,  91.,  99., 110.,  53., 131., 175.,
        139.,  43., 145., 157.,  55., 128.,  59., 146., 132.,  55.,  87.,
         25.,  93.,  32.,  76.,  75.,  23.,  86.,  95.,  98.,  20.,  77.,
         25.,  13.,  23.,  73.,  39.,  86.,  47.]),
 array([151., 183., 123., 176., 162., 125., 177., 147.,  78., 196., 187.,
        169.,  76., 179., 215., 118., 152.,  78., 203., 208., 132., 146.,
         59., 128.,  55., 157., 145.,  43., 139., 175., 131.,  53., 110.,
         99.,  91.,  80., 144.,  63., 167.,  69.]),
 array([170., 218., 201., 243., 202., 178., 212., 194., 175., 271., 273.,
        209., 167., 271., 279., 212., 175., 154., 290., 296., 208., 203.,
         78., 152., 118., 215., 179.,  76., 169., 187., 196.,  78., 14